In [1]:
import json
import importlib
cim_profile = 'rc4_2021'
cim = importlib.import_module('cimgraph.data_profile.' + cim_profile)

In [2]:
from cimgraph.databases import ConnectionParameters
from cimgraph.databases import GridappsdConnection
from cimgraph.models import FeederModel
# from cimgraph.models import DistributedArea

In [3]:
from test_get_all_edges import test_get_all_edges

In [4]:
params = ConnectionParameters(host = "localhost", port = "61613", cim_profile='rc4_2021', iec61970_301=7)
gapps = GridappsdConnection(params)

In [5]:
feeder_mrid = "_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"
feeder = cim.Feeder(mRID = feeder_mrid)
network = FeederModel(connection=gapps, container=feeder, distributed=False)


In [6]:
# feeder_mrids = ["_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62", # 13 bus
#                 "_C1C3E687-6FFD-C753-582B-632A27E28507",  # 123 bus
#                 "5B816B93-7A5F-B64C-8460-47C17D6E4B0F", # 13 bus asets
#                 "_4F76A5F9-271D-9EB8-5E31-AA362D86F2C3",  # 8500 node
#                 "_67AB291F-DCCD-31B7-B499-338206B9828F", # J1
#                 "_9CE150A8-8CC5-A0F9-B67E-BBD8C79D3095"]  # R2 12.47 3
# # feeder_mrid = "_EE71F6C9-56F0-4167-A14E-7F4C71F10EAA" #9500 node
# # feeder_mrid = "_9E985101-27AD-4FE4-B36A-EBECC98CDFAB" #MAPLE 10 bus
# for feeder_mrid in feeder_mrids:
#     feeder = cim.Feeder(mRID = feeder_mrid)
#     network = FeederModel(connection=gapps, container=feeder, distributed=False)
#     test_get_all_edges(network, cim)

In [7]:
with open("topo_message.json") as topo:
    topo_msg = json.load(topo)

In [8]:
# central_network = FeederModel(connection=gapps, container=feeder, distributed=False)

# dist_network_2 =  FeederModel(connection=gapps, container=feeder, distributed=True)

# dist_network_3 =  FeederModel(connection=gapps, container=feeder, distributed=True, topology_message=topo_msg)

In [9]:
# for switch_area in dist_network_2.distributed_areas:
#     test_get_all_edges(switch_area, cim)
#     for sec_area in switch_area.distributed_areas:
#         test_get_all_edges(sec_area, cim)

In [10]:
# for switch_area in dist_network_3.distributed_areas:
#     test_get_all_edges(switch_area, cim)
#     for sec_area in switch_area.distributed_areas:
#         test_get_all_edges(sec_area, cim)

In [11]:
from cimgraph.models import DistributedArea

container = cim.EquipmentContainer(mRID=feeder_mrid)
feeder_area = DistributedArea(container, gapps, distributed=True)
feeder_area.build_from_topo_message(topo_msg['feeders'])

feeder_area.get_all_edges(cim.ConnectivityNode)
for node in feeder_area.graph[cim.ConnectivityNode].values():
    print(node.name)

650
sourcebus
650z
brkr
645
632
633
rg60
646
xf1
634
tap
611
675
692
671
670
680
652
mid
684
house


In [13]:
counter = 0
for switch_topo in topo_msg['feeders']['switch_areas']:
    print('switch area', counter)
    container = cim.EquipmentContainer(mRID=feeder_mrid+'.'+str(counter))
    SwitchArea = DistributedArea(container, gapps, distributed=True)
    SwitchArea.build_from_topo_message(switch_topo)
    counter = counter + 1

    SwitchArea.get_all_edges(cim.ConnectivityNode)
    for node in SwitchArea.graph[cim.ConnectivityNode].values():
        print(node.name)

switch area 0
brkr
rg60
632
633
646
645
switch area 1
xf1
634
switch area 2
tap
611
switch area 3
675
692
switch area 4
mid
670
684
652
671
680
house


In [15]:
sw_counter = -1
for switch_topo in topo_msg['feeders']['switch_areas']:
    sw_counter +=1
    sa_counter = -1
    for sec_topo in switch_topo['secondary_areas']:
        sa_counter +=1
        area_id = str(sw_counter) + '.' + str(sa_counter)
        print('sec area', area_id)
        container = cim.EquipmentContainer(mRID=area_id)
        SecondaryArea = DistributedArea(container, gapps, distributed=True)
        SecondaryArea.build_from_topo_message(sec_topo)

        SecondaryArea.get_all_edges(cim.ConnectivityNode)
        for node in SecondaryArea.graph[cim.ConnectivityNode].values():
            print(node.name)

sec area 0.0
646
645
632
633
rg60
sec area 1.0
634
sec area 4.0
house
